https://github.com/VinAIResearch/PhoBERT

https://www.kaggle.com/datasets/gamind/uit-viquad-2-0

In [17]:
# -*- coding: utf-8 -*-
"""
Fine-tuning PhoBERT cho Hỏi-Đáp Tiếng Việt với UIT-ViQuAD 2.0
"""

# @title 1. Cài đặt và Import thư viện
# Cài các thư viện cần thiết từ Hugging Face
!pip install transformers datasets accelerate -q

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator # Sẽ dùng để tạo batch dữ liệu
)
from tqdm.auto import tqdm
import collections
import numpy as np

# Kiểm tra và sử dụng GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Thiết bị đang sử dụng: {device}")

Thiết bị đang sử dụng: cuda


# Giới thiệu Nhanh
Mô hình PhoBERT: Là một mô hình Transformer được VinAI Research huấn luyện trước (pre-trained) đặc biệt cho tiếng Việt. Nó có khả năng hiểu ngữ cảnh và ngữ nghĩa tiếng Việt tốt, phù hợp cho nhiều tác vụ NLP, bao gồm Hỏi-Đáp. Chúng ta sẽ sử dụng phiên bản vinai/phobert-base.

Dữ liệu UIT-ViQuAD 2.0: Là bộ dữ liệu Hỏi-Đáp Tiếng Việt do Đại học CNTT (UIT) phát triển, dựa trên định dạng SQuAD 2.0. Nó chứa các cặp (ngữ cảnh, câu hỏi, câu trả lời) lấy từ Wikipedia tiếng Việt. Điểm đặc biệt là bộ dữ liệu này có cả những câu hỏi có thể trả lời được và không thể trả lời được từ ngữ cảnh cho trước.

In [18]:
# @title 2. Tải Dữ liệu UIT-ViQuAD 2.0
# Tải dataset từ Hugging Face Hub
dataset_name = "taidng/UIT-ViQuAD2.0"
try:
    raw_datasets = load_dataset(dataset_name)
    print("\nTải dataset thành công:")
    print(raw_datasets)
    # In thử một mẫu
    print("\nMẫu dữ liệu (train):")
    print(raw_datasets["train"][10]) # In mẫu thứ 10
except Exception as e:
    print(f"Lỗi khi tải dataset: {e}")
    print("Vui lòng kiểm tra lại tên dataset hoặc kết nối mạng.")


Tải dataset thành công:
DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 28454
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

Mẫu dữ liệu (train):
{'id': '0001-0002-0004', 'uit_id': 'uit_000011', 'title': 'Phạm Văn Đồng', 'context': 'Năm 1954, ông được giao nhiệm vụ Trưởng phái đoàn Chính phủ dự Hội nghị Genève về Đông Dương. Những đóng góp của đoàn Việt Nam do ông đứng đầu là vô cùng quan trọng, tạo ra những đột phá đưa Hội nghị tới thành công. Trải qua 8 phiên họp toàn thể và 23 phiên họp rất căng thẳng và phức tạp, với tinh thần chủ động và cố gắng của phái đoàn Vi

In [23]:
# @title 3. Tải Model PhoBERT và Tokenizer
# Tải tokenizer và model PhoBERT đã pre-trained cho tác vụ Question Answering
# model_checkpoint = "vinai/phobert-base"

# @title 3. Tải Model XLM-RoBERTa và Tokenizer
# Tải tokenizer và model XLM-RoBERTa đã pre-trained cho tác vụ Question Answering
model_checkpoint = "xlm-roberta-base" # <<<=== THAY ĐỔI Ở ĐÂY
try:
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True) # Vẫn giữ use_fast=True
    model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint).to(device)
    print(f"\nTải model {model_checkpoint} và tokenizer thành công.")

    # === THÊM 2 DÒNG NÀY ĐỂ KIỂM TRA ===
    print(f"Loại Tokenizer: {type(tokenizer)}")
    print(f"Là Fast Tokenizer: {tokenizer.is_fast}")
    # ==================================

except Exception as e:
    print(f"Lỗi khi tải model hoặc tokenizer: {e}")
    print("Vui lòng kiểm tra lại tên model checkpoint.")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Tải model xlm-roberta-base và tokenizer thành công.
Loại Tokenizer: <class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>
Là Fast Tokenizer: True


In [26]:
# @title 4. Tiền xử lý Dữ liệu (Quan trọng) - PHIÊN BẢN ĐÃ SỬA LỖI

# Các tham số cho việc tiền xử lý
max_length = 384
doc_stride = 128

def prepare_features(examples):
    # Thêm khoảng trắng trước câu hỏi
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize context và question
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Thêm các trường label mới
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # Kiểm tra xem trường 'answers' có tồn tại trong batch này không
    # Giả định rằng nếu không có 'answers' thì đó là test set
    answers_exist = "answers" in examples and examples["answers"] is not None

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]

        # Đặt giá trị mặc định (cho test set hoặc câu hỏi không trả lời được)
        start_position = cls_index
        end_position = cls_index

        # Chỉ xử lý tìm vị trí start/end nếu 'answers' tồn tại
        if answers_exist and sample_index < len(examples["answers"]): # Thêm kiểm tra sample_index
            answers = examples["answers"][sample_index]
             # Kiểm tra xem 'is_impossible' có tồn tại không (có thể có hoặc không trong test set thật)
            is_impossible = examples.get("is_impossible", [True] * len(examples["context"]))[sample_index] # Lấy flag, mặc định True nếu thiếu

            # Chỉ tìm vị trí nếu câu hỏi có thể trả lời được và có câu trả lời
            if answers and not is_impossible and len(answers["answer_start"]) > 0:
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])

                token_start_index = 0
                while sequence_ids[token_start_index] != 1:
                    token_start_index += 1
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != 1:
                    token_end_index -= 1

                # Kiểm tra câu trả lời có nằm trong span không
                if offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char:
                    # Tìm token start
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    start_position = token_start_index - 1
                    # Tìm token end
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    end_position = token_end_index + 1
                # else: không nằm trong span, giữ nguyên start/end là cls_index

        # Lưu kết quả vị trí start/end
        tokenized_examples["start_positions"].append(start_position)
        tokenized_examples["end_positions"].append(end_position)

    return tokenized_examples


# Áp dụng hàm tiền xử lý vào toàn bộ dataset
print("\nBắt đầu tiền xử lý dataset...")
try:
    # Xóa cột 'answers' và 'is_impossible' trước khi map nếu chúng gây lỗi (chỉ áp dụng cho test set nếu có)
    # Tuy nhiên, cách tốt hơn là xử lý trong hàm map như trên
    tokenized_datasets = raw_datasets.map(
        prepare_features,
        batched=True,
        remove_columns=raw_datasets["train"].column_names # Vẫn xóa cột cũ để gọn gàng
    )
    print("\nTiền xử lý hoàn tất.")
    print("Dataset sau tiền xử lý:")
    print(tokenized_datasets)
except Exception as e:
    print(f"\nLỗi trong quá trình tiền xử lý: {e}")


Bắt đầu tiền xử lý dataset...


Map:   0%|          | 0/28454 [00:00<?, ? examples/s]

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

Map:   0%|          | 0/7301 [00:00<?, ? examples/s]


Tiền xử lý hoàn tất.
Dataset sau tiền xử lý:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 30399
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3937
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 7695
    })
})


In [27]:
# @title 5. Huấn luyện Mô hình (Fine-tuning)

# Định nghĩa nơi lưu model và các tham số huấn luyện
output_dir = "phobert-finetuned-viquad2"
# Giảm batch_size nếu gặp lỗi Out-of-Memory (OOM) trên Colab
# Tăng số epochs nếu muốn huấn luyện kỹ hơn (nhưng tốn thời gian hơn)
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch", # Đánh giá sau mỗi epoch
    learning_rate=3e-5,          # Learning rate phổ biến cho fine-tuning BERT
    per_device_train_batch_size=8, # Giảm nếu cần (vd: 4)
    per_device_eval_batch_size=8,  # Giảm nếu cần
    num_train_epochs=2,          # Số epoch huấn luyện (tăng lên 3 hoặc hơn nếu cần)
    weight_decay=0.01,
    save_strategy="epoch",       # Lưu model sau mỗi epoch
    load_best_model_at_end=True, # Tải model tốt nhất sau khi huấn luyện xong
    # push_to_hub=False,         # Đặt True nếu muốn đẩy lên Hugging Face Hub
)

# Khởi tạo Trainer
# default_data_collator sẽ tự động tạo batch từ dataset đã xử lý
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"], # Sử dụng tập validation để đánh giá
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

# Bắt đầu huấn luyện
# Quá trình này có thể mất thời gian đáng kể, phụ thuộc vào GPU của Colab
print("\nBắt đầu quá trình fine-tuning...")
try:
    trainer.train()
    print("\nHuấn luyện hoàn tất!")
except Exception as e:
    print(f"\nLỗi trong quá trình huấn luyện: {e}")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-27-c2305ad013ba>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



Bắt đầu quá trình fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anime365247-aaa (anime365247-aaa-tuy-n-sinh-h-spkt-tphcm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.544100,1.555712
2,1.220800,1.451755



Huấn luyện hoàn tất!


In [28]:
# @title 6. Lưu Model đã Huấn luyện
# Lưu model và tokenizer tốt nhất vào thư mục output_dir
print(f"\nLưu model và tokenizer vào thư mục: {output_dir}")
try:
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    print("Lưu thành công!")
    print(f"Bạn có thể tìm thấy model trong thư mục '{output_dir}' ở cây thư mục bên trái của Colab.")
except Exception as e:
    print(f"Lỗi khi lưu model: {e}")


# @title 7. (Tùy chọn) Sử dụng Model để Dự đoán
# Ví dụ cách dùng model đã fine-tune để trả lời câu hỏi

from transformers import pipeline

# Tải model đã lưu (nếu cần chạy lại session)
# saved_model_path = output_dir
# qa_pipeline = pipeline("question-answering", model=saved_model_path, tokenizer=saved_model_path, device=0 if torch.cuda.is_available() else -1)

# Hoặc sử dụng model đang có trong bộ nhớ (nếu vừa huấn luyện xong)
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


# Ví dụ
context = "Trường Đại học Công nghệ Thông tin (UIT) là một trường đại học thành viên của Đại học Quốc gia Thành phố Hồ Chí Minh, được thành lập vào ngày 8 tháng 6 năm 2006. Trường chuyên đào tạo về lĩnh vực công nghệ thông tin và truyền thông."
question = "UIT được thành lập khi nào?"

print("\nVí dụ dự đoán:")
print(f"Context: {context}")
print(f"Question: {question}")

try:
    result = qa_pipeline(question=question, context=context)
    print("\nKết quả dự đoán:")
    print(f"Answer: {result['answer']}")
    print(f"Score: {result['score']:.4f}")
    print(f"Start: {result['start']}")
    print(f"End: {result['end']}")
except Exception as e:
     print(f"\nLỗi khi dự đoán: {e}")

# Ví dụ câu hỏi không có câu trả lời trong context
question_impossible = "Ai là hiệu trưởng đầu tiên của UIT?"
print(f"\nQuestion (Impossible): {question_impossible}")
try:
    result_impossible = qa_pipeline(question=question_impossible, context=context)
    print("\nKết quả dự đoán (Impossible):")
     # Model tốt sẽ trả về score thấp hoặc chuỗi rỗng (tùy cách xử lý)
    print(f"Answer: {result_impossible['answer']}")
    print(f"Score: {result_impossible['score']:.4f}")
except Exception as e:
     print(f"\nLỗi khi dự đoán: {e}")


Lưu model và tokenizer vào thư mục: phobert-finetuned-viquad2


Device set to use cuda:0


Lưu thành công!
Bạn có thể tìm thấy model trong thư mục 'phobert-finetuned-viquad2' ở cây thư mục bên trái của Colab.

Ví dụ dự đoán:
Context: Trường Đại học Công nghệ Thông tin (UIT) là một trường đại học thành viên của Đại học Quốc gia Thành phố Hồ Chí Minh, được thành lập vào ngày 8 tháng 6 năm 2006. Trường chuyên đào tạo về lĩnh vực công nghệ thông tin và truyền thông.
Question: UIT được thành lập khi nào?

Kết quả dự đoán:
Answer: ngày 8 tháng 6 năm 2006.
Score: 0.6327
Start: 137
End: 161

Question (Impossible): Ai là hiệu trưởng đầu tiên của UIT?

Kết quả dự đoán (Impossible):
Answer: Đại học Quốc gia Thành phố Hồ Chí Minh,
Score: 0.0576


In [29]:
# @title 7. Đóng gói và Tải Model (Zip and Download Model)

import shutil
from google.colab import files
import os

# --- Lấy tên thư mục đã lưu model ---
# Đảm bảo biến `output_dir` đã được định nghĩa ở Cell 5 và chứa đúng tên thư mục
try:
    # Tên thư mục chứa model đã lưu (ví dụ: "phobert-finetuned-viquad2" hoặc "xlm-roberta-finetuned-viquad2")
    saved_model_directory = output_dir

    print(f"[*] Thư mục chứa model cần đóng gói: '{saved_model_directory}'")

    # Kiểm tra xem thư mục có thực sự tồn tại không
    if os.path.isdir(saved_model_directory):
        # --- Tạo tên file zip (giống tên thư mục) ---
        zip_filename_base = saved_model_directory
        zip_filepath = f"{zip_filename_base}.zip" # Tên file zip cuối cùng

        print(f"[*] Đang nén thư mục '{saved_model_directory}' thành file '{zip_filepath}'...")

        # --- Thực hiện nén thư mục ---
        # Cú pháp: shutil.make_archive(tên_file_nén_không_đuôi, 'định_dạng', thư_mục_gốc_cần_nén)
        shutil.make_archive(zip_filename_base, 'zip', saved_model_directory)

        print(f"[*] Nén thành công! File '{zip_filepath}' đã được tạo.")

        # --- Kích hoạt tải file zip về máy tính ---
        print(f"[*] Chuẩn bị tải file '{zip_filepath}' về máy...")
        files.download(zip_filepath)
        print("[*] Hoàn tất! Kiểm tra cửa sổ download của trình duyệt.")

    else:
        print(f"[!] Lỗi: Không tìm thấy thư mục '{saved_model_directory}'.")
        print("    Hãy chắc chắn rằng bạn đã chạy Cell 6 (Lưu Model) thành công.")

except NameError:
    # Xử lý trường hợp biến output_dir chưa được tạo
    print("[!] Lỗi: Không tìm thấy biến 'output_dir'.")
    print("    Vui lòng kiểm tra lại Cell 5 nơi bạn định nghĩa `TrainingArguments` và đảm bảo nó đã chạy.")
except Exception as e:
    # Bắt các lỗi khác có thể xảy ra
    print(f"[!] Có lỗi xảy ra trong quá trình đóng gói hoặc tải về: {e}")

[*] Thư mục chứa model cần đóng gói: 'phobert-finetuned-viquad2'
[*] Đang nén thư mục 'phobert-finetuned-viquad2' thành file 'phobert-finetuned-viquad2.zip'...
[*] Nén thành công! File 'phobert-finetuned-viquad2.zip' đã được tạo.
[*] Chuẩn bị tải file 'phobert-finetuned-viquad2.zip' về máy...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[*] Hoàn tất! Kiểm tra cửa sổ download của trình duyệt.
